### P2: Baseball Managers
Thomas Hrabchak <br>
December 2015

### Introduction
Major League Baseball (MLB) is known for its all-stars making multi million dollar salaries and hitting towering homeruns. Often overlooked is the manager, who rarely leaves the duggout except to argue with an umpire, throw his hat, and get ejected from the game. Behind the scenes there are many decisions being made by the manager every pitch, such as the positioning of fielders, preparation of a relief pitcher, or if the batter should try to bunt. Some managers are well known, have winning teams, and make it to the hall of fame, many others are forgotten by fans. This project will explore the role of the manager on a Major League Baseball team.

### Questions
- Which, if any, team statistics correlate with the winningest managers?
- If a manager transfers teams, do any statistics from his previous team move to the new team?
- What are characteristics of managers that make it into the hall of fame?
- What are characteristics of managers that have short careers?
- Are any managers particularly good in the post season?
- Do any team statistics correlate with a long managerial career?

- Most recent player manager?
- Salaries of managers?

### Wrangle

- how to get the teams that a manager manages?
- possibly limit data to managers that serve a full year with one team?

- acquisition
- data cleaning

### Explore
- build intuition
- find patterns

### Concluison
- answers to questions based on exploration

### References

### Appendix
